## Little pandas code to get UK holidays between 2000 and 2022 (aas our traffic data)

### API REFERENCE: https://date.nager.at/Api

In [1]:
import pandas as pd;

In [2]:
holidays = pd.DataFrame()

for x in range(2000, 2023):
    ##Transformations to adjust our data
    new = pd.read_json(f'https://date.nager.at/api/v3/PublicHolidays/{x}/GB')
    new[["name", "counties"]] = new[["name", "counties"]].astype(str).apply(lambda x : x.str.upper())
    new["date"] = new["date"].dt.date
    new["holiday_date_id"] = new["date"].astype(str).str.replace('-', '').astype(int)
    new["counties"] = new["counties"].str.replace("'GB-NIR'", "NORTHERN IRELAND").\
                                  str.replace("'GB-ENG', 'GB-WLS'", "ENGLAND AND WALES").\
                                  str.replace("'GB-ENG'", "ENGLAND AND WALES").\
                                  str.replace("'GB-WLS'", "ENGLAND AND WALES").\
                                  str.replace("'GB-SCT'", "SCOTLAND").\
                                  str.replace('[', '', regex=False).\
                                  str.replace(']', '', regex=False).\
                                  str.replace('NONE', 'GLOBAL').\
                                  str.replace(", NORTHERN IRELAND", "").\
                                  str.replace("ENGLAND AND WALES, SCOTLAND", "GLOBAL") ##If it exists, we will consider it as GLOBAL
    new.rename(columns={"counties" : "holiday_region", "name":"holiday_name", "date":"holiday_date"}, inplace=True)
    new = new[["holiday_date", "holiday_region", "holiday_name", "holiday_date_id"]]
    ## We will discard "NORTHERN IRELAND" holidays, because we don't have traffic data from that county.
    new = new[new["holiday_region"] != 'NORTHERN IRELAND']
    holidays = pd.concat([holidays, new])

In [3]:
holidays

,holiday_date,holiday_region,holiday_name,holiday_date_id
1,2000-01-03,ENGLAND AND WALES,NEW YEAR'S DAY,20000103
2,2000-01-03,SCOTLAND,NEW YEAR'S DAY,20000103
3,2000-01-04,SCOTLAND,NEW YEAR'S DAY,20000104
5,2000-04-21,GLOBAL,GOOD FRIDAY,20000421
6,2000-04-24,ENGLAND AND WALES,EASTER MONDAY,20000424
...,...,...,...,...
12,2022-08-29,ENGLAND AND WALES,SUMMER BANK HOLIDAY,20220829
13,2022-09-19,GLOBAL,QUEEN’S STATE FUNERAL,20220919
14,2022-11-30,SCOTLAND,SAINT ANDREW'S DAY,20221130
15,2022-12-26,GLOBAL,ST. STEPHEN'S DAY,20221226


In [4]:
holidays.to_parquet('../../../01_aux_files/source/UK_Traffic_Data/uk_holiday/uk_holidays.parquet')

In [5]:
print("Great!")

Great!
